# Modelling of Agersens Data with Downsampling

### Importing the required libraries

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import math
import numpy as np
from scipy import stats

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, classification_report

import xgboost as xg

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU

### Reading data from the data provided using pandas

In [2]:
agersens_df = pd.read_csv('/data/shared/debdeep_guha/agersens_data/location.csv')

In [3]:
agersens_df.head()

,ID,target,latitude,longitude,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,Location_1,1,-37.706060,145.126555,6,1.5,9.0,7.33,43.98,10.995,2,12,3.0,14.66,999.62,17.48
1,Location_2,1,-37.706120,145.126457,11,0.8,8.8,3.52,38.72,2.816,3,33,2.4,10.56,1000.17,22.59
2,Location_3,1,-37.706122,145.126435,12,0.7,8.4,2.93,35.16,2.051,3,36,2.1,8.79,986.20,21.66
3,Location_4,1,-37.706123,145.126427,13,0.7,9.1,2.54,33.02,1.778,4,52,2.8,10.16,1011.47,13.72
4,Location_5,1,-37.706123,145.126423,13,0.7,9.1,2.29,29.77,1.603,4,52,2.8,9.16,995.29,9.19


In [4]:
# checking the length of the dataset
len(agersens_df)

4106295

#### Checking the data

In [5]:
agersens_df.describe()

,target,latitude,longitude,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
count,4.106295e+06,3.877678e+06,4.106295e+06,4.106295e+06,4.106295e+06,3.101952e+06,4.106295e+06,4.073446e+06,4.106295e+06,4.106295e+06,4.106295e+06,4.106295e+06,4.098033e+06,4.106295e+06,4.106295e+06
mean,9.919292e-01,-3.770612e+01,1.451264e+02,1.508246e+01,7.535538e-01,1.031854e+01,1.866368e+00,1.962658e+01,2.419367e+00,2.868143e+02,5.175781e+03,1.716860e+02,1.582932e+02,9.999998e+02,1.999491e+01
std,8.947426e-02,8.390644e-05,1.281560e-04,4.389738e+00,4.288497e-01,2.702694e+00,6.515952e+00,3.618669e+01,3.508119e+01,2.959106e+02,5.502739e+03,1.779611e+02,1.550168e+02,1.000328e+01,4.998535e+00
min,0.000000e+00,-3.773290e+01,1.450800e+02,3.000000e+00,4.000000e-01,3.000000e+00,4.800000e-01,3.480000e+00,1.960000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.514000e+02,-6.250000e+00
25%,1.000000e+00,-3.770613e+01,1.451264e+02,1.100000e+01,6.000000e-01,9.500000e+00,5.000000e-01,9.000000e+00,3.000000e-01,1.000000e+01,1.100000e+02,9.500000e+00,2.493000e+01,9.932400e+02,1.662000e+01
50%,1.000000e+00,-3.770612e+01,1.451264e+02,1.600000e+01,7.000000e-01,1.050000e+01,5.800000e-01,1.080000e+01,3.500000e-01,1.850000e+02,3.222000e+03,1.092000e+02,1.020800e+02,1.000000e+03,1.999000e+01
75%,1.000000e+00,-3.770611e+01,1.451264e+02,1.900000e+01,8.000000e-01,1.100000e+01,2.050000e+00,2.250000e+01,1.792000e+00,5.420000e+02,9.690000e+03,3.150000e+02,2.800000e+02,1.006750e+03,2.337000e+01
max,1.000000e+00,-3.762418e+01,1.452322e+02,6.300000e+01,1.117000e+02,1.408000e+03,6.549600e+02,9.410550e+03,2.402215e+04,9.010000e+02,2.157600e+04,3.449600e+04,4.812372e+04,1.049120e+03,4.540000e+01


In [6]:
agersens_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4106295 entries, 0 to 4106294
Data columns (total 16 columns):
 #   Column     Dtype  
---  ------     -----  
 0   ID         object 
 1   target     int64  
 2   latitude   float64
 3   longitude  float64
 4   f1         int64  
 5   f2         float64
 6   f3         float64
 7   f4         float64
 8   f5         float64
 9   f6         float64
 10  f7         int64  
 11  f8         int64  
 12  f9         float64
 13  f10        float64
 14  f11        float64
 15  f12        float64
dtypes: float64(11), int64(4), object(1)
memory usage: 501.3+ MB


No categorical column present.

In [7]:
agersens_df.isnull().sum()

ID                 0
target             0
latitude      228617
longitude          0
f1                 0
f2                 0
f3           1004343
f4                 0
f5             32849
f6                 0
f7                 0
f8                 0
f9                 0
f10             8262
f11                0
f12                0
dtype: int64

Nulls in `latitude`, `f3`, `f5`, `f10`. 

Removing rows where `latitude` is null. As cannot impute values for geographical coordinates and we will only be losing 246 rows for `Non-Spurious` data, which is less

In [8]:
agersens_df['target'].unique()

array([1, 0])

In [9]:
agersens_df[agersens_df['latitude'].isnull()]

,ID,target,latitude,longitude,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
35,Location_36,1,NaN,145.126405,15,0.6,9.0,1.09,16.35,0.654,20,300,12.0,21.80,987.39,13.24
36,Location_37,1,NaN,145.126405,15,0.6,9.0,1.07,16.05,0.642,20,300,12.0,21.40,1024.25,21.07
37,Location_38,1,NaN,145.126405,15,0.6,9.0,1.06,15.90,0.636,21,315,12.6,22.26,999.15,24.53
38,Location_39,1,NaN,145.126405,15,0.6,9.0,1.05,15.75,0.630,21,315,12.6,22.05,1012.19,18.90
39,Location_40,1,NaN,145.126405,15,0.6,9.0,1.04,15.60,0.624,22,330,13.2,22.88,979.04,21.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4105830,Location_4105831,1,NaN,145.126300,8,1.2,NaN,4.83,38.64,5.796,2,16,2.4,9.66,1006.41,23.38
4105831,Location_4105832,1,NaN,145.126300,11,0.9,9.9,4.10,45.10,3.690,3,33,2.7,12.30,998.45,20.13
4106170,Location_4106171,1,NaN,145.126400,15,0.7,10.5,0.97,14.55,0.679,60,900,42.0,58.20,978.37,22.41
4106171,Location_4106172,1,NaN,145.126400,15,0.7,10.5,0.97,14.55,0.679,61,915,42.7,59.17,1000.23,20.01


In [10]:
len(agersens_df) - len(agersens_df[agersens_df['latitude'].isnull()])

3877678

#### Only 0.8% of the data is `Non-Spurious`

The data is highly imbalanced, we have to be careful in removing records

In [11]:
agersens_df['target'].value_counts()

1    4073154
0      33141
Name: target, dtype: int64

In [12]:
agersens_df['target'].value_counts(normalize=True)

1    0.991929
0    0.008071
Name: target, dtype: float64

#### Imputing missing values for `f3` with the mean value

In [13]:
agersens_df['f3'] = agersens_df['f3'].fillna(agersens_df['f3'].mean())

In [14]:
agersens_df.isnull().sum()

ID                0
target            0
latitude     228617
longitude         0
f1                0
f2                0
f3                0
f4                0
f5            32849
f6                0
f7                0
f8                0
f9                0
f10            8262
f11               0
f12               0
dtype: int64

In [15]:
len(agersens_df)

4106295

#### Removing rows where null value is present in any column

In [16]:
agersens_df = agersens_df.dropna(axis=0, how="any")

In [17]:
agersens_df.isnull().sum()

ID           0
target       0
latitude     0
longitude    0
f1           0
f2           0
f3           0
f4           0
f5           0
f6           0
f7           0
f8           0
f9           0
f10          0
f11          0
f12          0
dtype: int64

In [18]:
len(agersens_df)

3839172

In [19]:
agersens_df['target'].value_counts()

1    3806293
0      32879
Name: target, dtype: int64

In [20]:
agersens_df.columns

Index(['ID', 'target', 'latitude', 'longitude', 'f1', 'f2', 'f3', 'f4', 'f5',
       'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12'],
      dtype='object')

### Checking the number of rows for each column beyond a threshold value using z-score

Keeping records less than 3rd standard deviation from columns: `f1`, `f9`, `f11` and `f12`

This is done based on the analysis done in `agersens_data_analysis.ipynb` notebook

In [21]:
upd_agersens_df = agersens_df.copy()

In [22]:
# keeping records which lie within the 3rd standard deviation using z-score
upd_agersens_df = upd_agersens_df[(np.abs(stats.zscore(upd_agersens_df['f1'])) < 3)]
upd_agersens_df = upd_agersens_df[(np.abs(stats.zscore(upd_agersens_df['f9'])) < 3)]
upd_agersens_df = upd_agersens_df[(np.abs(stats.zscore(upd_agersens_df['f11'])) < 3)]
upd_agersens_df = upd_agersens_df[(np.abs(stats.zscore(upd_agersens_df['f12'])) < 3)]

In [23]:
len(upd_agersens_df)

3815118

In [24]:
upd_agersens_df['target'].value_counts()

1    3782409
0      32709
Name: target, dtype: int64

##### creating a new column combining the latitude and longitude columns

(Might use it as a categorical column, will remove it otherwise)

In [25]:
upd_agersens_df['coordinates'] = upd_agersens_df['latitude'].astype(str) + ", " + upd_agersens_df['longitude'].astype(str)

In [26]:
upd_agersens_df.head()

,ID,target,latitude,longitude,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,coordinates
0,Location_1,1,-37.706060,145.126555,6,1.5,9.0,7.33,43.98,10.995,2,12,3.0,14.66,999.62,17.48,"-37.70606, 145.126555"
1,Location_2,1,-37.706120,145.126457,11,0.8,8.8,3.52,38.72,2.816,3,33,2.4,10.56,1000.17,22.59,"-37.70612, 145.126456667"
2,Location_3,1,-37.706122,145.126435,12,0.7,8.4,2.93,35.16,2.051,3,36,2.1,8.79,986.20,21.66,"-37.706121667, 145.12643500000001"
3,Location_4,1,-37.706123,145.126427,13,0.7,9.1,2.54,33.02,1.778,4,52,2.8,10.16,1011.47,13.72,"-37.706123333, 145.126426667"
4,Location_5,1,-37.706123,145.126423,13,0.7,9.1,2.29,29.77,1.603,4,52,2.8,9.16,995.29,9.19,"-37.706123333, 145.126423333"


In [27]:
len(upd_agersens_df)

3815118

In [28]:
len(upd_agersens_df['coordinates'].unique())

4431

### Feature Engineering

In [29]:
# creating a new dataframe and dropping the ID and coordinates column
full_upd_agersens_df = upd_agersens_df.drop(['ID', 'coordinates'], axis = 1)

In [30]:
full_upd_agersens_df.columns

Index(['target', 'latitude', 'longitude', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6',
       'f7', 'f8', 'f9', 'f10', 'f11', 'f12'],
      dtype='object')

#### Scaling certain columns using MinMax Scaler

Using MinMax Scaler as either I have removed outliers from them or there were none to begin with.

In [31]:
minmax_scaler = MinMaxScaler()

minmax_columns = ['f1', 'f7', 'f8', 'f9', 'f11', 'f12']

full_upd_agersens_df[minmax_columns] = minmax_scaler.fit_transform(full_upd_agersens_df[minmax_columns])

#### Scaling certain columns using Robust Scaler

Using Robust scaler as there were certain columns where removing rows with outliers can lead to losing out on a lot of `Non-Spurious` data, this will consider outliers during scaling

In [32]:
robust_scaler = RobustScaler()

robust_columns = ['f2', 'f3', 'f4', 'f5', 'f6', 'f10']

full_upd_agersens_df[robust_columns] = robust_scaler.fit_transform(full_upd_agersens_df[robust_columns])

In [33]:
full_upd_agersens_df.head()

,target,latitude,longitude,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,1,-37.706060,145.126555,0.142857,4.0,-1.318543,4.299363,2.404364,7.012516,0.00222,0.000556,0.004274,-0.332224,0.493669,0.416139
1,1,-37.706120,145.126457,0.380952,0.5,-1.518543,1.872611,2.021818,1.624506,0.00333,0.001529,0.003419,-0.348470,0.502832,0.586529
2,1,-37.706122,145.126435,0.428571,0.0,-1.918543,1.496815,1.762909,1.120553,0.00333,0.001669,0.002991,-0.355484,0.270077,0.555519
3,1,-37.706123,145.126427,0.476190,0.0,-1.218543,1.248408,1.607273,0.940711,0.00444,0.002410,0.003989,-0.350055,0.691103,0.290764
4,1,-37.706123,145.126423,0.476190,0.0,-1.218543,1.089172,1.370909,0.825428,0.00444,0.002410,0.003989,-0.354018,0.421526,0.139713


### Generating metrics after downsampling the data and running different models and generating metrics for different random states

Generating metrics like `Accuracy`, `F1Score` and `Confusion Matrix` for different random states, to see the reliability of the model

### Support Vector Machine (SVM) Classifier

In [34]:
svc_random_state_df = pd.DataFrame(
    {'run_id': list(range(0, 10)),
     'random_state': list(range(0, 10))
    })

In [35]:
train_accuracy_list = []
test_accuracy_list = []
train_f1_list = []
test_f1_list = []
train_true_pos_list = []
train_false_pos_list = []
train_false_neg_list = []
train_true_neg_list = []
test_true_pos_list = []
test_false_pos_list = []
test_false_neg_list = []
test_true_neg_list = []


for j in range(10):
    
    shufled_agersens_df = full_upd_agersens_df.sample(frac=1, random_state=j).reset_index(drop=True)
    
    X = shufled_agersens_df.drop(columns=['target'])
    y = shufled_agersens_df['target']

    #split data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=j)

    #training part
    #combine them back for resampling
    train_data = pd.concat([X_train, y_train], axis=1)
    
    # separate minority and majority classes
    negative = train_data[train_data['target']==1]
    positive = train_data[train_data['target']==0]

    # downsample majority
    neg_downsampled = resample(negative,
     replace=True, # sample with replacement
     n_samples=len(positive), # match number in minority class
     random_state=j) # reproducible results
    # combine minority and downsampled majority
    train_downsampled = pd.concat([positive, neg_downsampled])

    #testing part
    #combine them back for resampling
    test_data = pd.concat([X_test, y_test], axis=1)
    
    # separate minority and majority classes
    negative = test_data[test_data['target']==1]
    positive = test_data[test_data['target']==0]

    # downsample majority
    neg_downsampled = resample(negative,
     replace=True, # sample with replacement
     n_samples=len(positive), # match number in minority class
     random_state=j) # reproducible results
    # combine minority and downsampled majority
    test_downsampled = pd.concat([positive, neg_downsampled])


    X_train = train_downsampled.drop(['target'],axis=1)

    X_test =  test_downsampled.drop(['target'],axis=1)

    y_train = train_downsampled['target']

    y_test = test_downsampled['target']

    svc_model = SVC(kernel='sigmoid')

    svc_model.fit(X_train, y_train)

    svc_train_preds = svc_model.predict(X_train)
    svc_test_preds = svc_model.predict(X_test)

    train_acc_svc = accuracy_score(y_train, svc_train_preds)
    test_acc_svc = accuracy_score(y_test, svc_test_preds)
    
    train_f1_svc = f1_score(y_train, svc_train_preds)
    test_f1_svc = f1_score(y_test, svc_test_preds)

    train_conf_matrix_svc = confusion_matrix(y_train, svc_train_preds)
    train_true_pos = train_conf_matrix_svc[0][0]
    train_false_pos = train_conf_matrix_svc[0][1]
    train_false_neg = train_conf_matrix_svc[1][0]
    train_true_neg = train_conf_matrix_svc[1][1]
    
    test_conf_matrix_svc = confusion_matrix(y_test, svc_test_preds)
    test_true_pos = test_conf_matrix_svc[0][0]
    test_false_pos = test_conf_matrix_svc[0][1]
    test_false_neg = test_conf_matrix_svc[1][0]
    test_true_neg = test_conf_matrix_svc[1][1]   
    
    train_accuracy_list.append(train_acc_svc)
    test_accuracy_list.append(test_acc_svc)
    
    train_f1_list.append(train_f1_svc)
    test_f1_list.append(test_f1_svc)
    
    train_true_pos_list.append(train_true_pos)
    train_false_pos_list.append(train_false_pos)    
    train_false_neg_list.append(train_false_neg)
    train_true_neg_list.append(train_true_neg)
    
    test_true_pos_list.append(test_true_pos)
    test_false_pos_list.append(test_false_pos)    
    test_false_neg_list.append(test_false_neg)
    test_true_neg_list.append(test_true_neg)
    
svc_random_state_df['train_accuracy'] = train_accuracy_list
svc_random_state_df['test_accuracy'] = test_accuracy_list
svc_random_state_df['train_f1_score'] = train_f1_list
svc_random_state_df['test_f1_score'] = test_f1_list
svc_random_state_df['train_true_pos'] = train_true_pos_list
svc_random_state_df['train_false_pos'] = train_false_pos_list
svc_random_state_df['train_false_neg'] = train_false_neg_list
svc_random_state_df['train_true_neg'] = train_true_neg_list
svc_random_state_df['test_true_pos'] = test_true_pos_list
svc_random_state_df['test_false_pos'] = test_false_pos_list
svc_random_state_df['test_false_neg'] = test_false_neg_list
svc_random_state_df['test_true_neg'] = test_true_neg_list

In [36]:
svc_random_state_df

,run_id,random_state,train_accuracy,test_accuracy,train_f1_score,test_f1_score,train_true_pos,train_false_pos,train_false_neg,train_true_neg,test_true_pos,test_false_pos,test_false_neg,test_true_neg
0,0,0,0.926961,0.924859,0.929362,0.927700,20515,2459,897,22077,8621,1114,349,9386
1,1,1,0.935658,0.935260,0.935672,0.935191,21357,1474,1464,21367,9249,629,650,9228
2,2,2,0.933728,0.935024,0.933732,0.935048,21393,1520,1517,21396,9156,640,633,9163
3,3,3,0.934320,0.936520,0.934338,0.936451,21495,1518,1505,21508,9091,605,626,9070
4,4,4,0.934541,0.936602,0.934516,0.936550,21467,1494,1512,21449,9138,610,626,9122
5,5,5,0.937377,0.931791,0.937360,0.931925,21411,1424,1436,21399,9181,693,654,9220
6,6,6,0.927362,0.926079,0.929771,0.928624,20458,2450,878,22030,8727,1074,375,9426
7,7,7,0.934265,0.935494,0.934216,0.935709,21350,1484,1518,21316,9205,670,604,9271
8,8,8,0.935744,0.934386,0.935733,0.934500,21484,1471,1479,21476,9097,657,623,9131
9,9,9,0.932910,0.936148,0.932893,0.936306,21462,1537,1549,21450,9066,644,596,9114


### Random Forest Classifier

In [37]:
rfc_random_state_df = pd.DataFrame(
    {'run_id': list(range(0, 10)),
     'random_state': list(range(0, 10))
    })

In [38]:
train_accuracy_list = []
test_accuracy_list = []
train_f1_list = []
test_f1_list = []
train_true_pos_list = []
train_false_pos_list = []
train_false_neg_list = []
train_true_neg_list = []
test_true_pos_list = []
test_false_pos_list = []
test_false_neg_list = []
test_true_neg_list = []


for j in range(10):
    
    shufled_agersens_df = full_upd_agersens_df.sample(frac=1, random_state=j).reset_index(drop=True)
    
    X = shufled_agersens_df.drop(columns=['target'])
    y = shufled_agersens_df['target']

    #split data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=j)

    #training part
    #combine them back for resampling
    train_data = pd.concat([X_train, y_train], axis=1)
    
    # separate minority and majority classes
    negative = train_data[train_data['target']==1]
    positive = train_data[train_data['target']==0]

    # downsample majority
    neg_downsampled = resample(negative,
     replace=True, # sample with replacement
     n_samples=len(positive), # match number in minority class
     random_state=j) # reproducible results
    # combine minority and downsampled majority
    train_downsampled = pd.concat([positive, neg_downsampled])

    #testing part
    #combine them back for resampling
    test_data = pd.concat([X_test, y_test], axis=1)
    
    # separate minority and majority classes
    negative = test_data[test_data['target']==1]
    positive = test_data[test_data['target']==0]

    # downsample majority
    neg_downsampled = resample(negative,
     replace=True, # sample with replacement
     n_samples=len(positive), # match number in minority class
     random_state=j) # reproducible results
    # combine minority and downsampled majority
    test_downsampled = pd.concat([positive, neg_downsampled])


    X_train = train_downsampled.drop(['target'],axis=1)

    X_test =  test_downsampled.drop(['target'],axis=1)

    y_train = train_downsampled['target']

    y_test = test_downsampled['target']

    rfc_model = RandomForestClassifier(max_depth = 12, max_features = 'sqrt')

    rfc_model.fit(X_train, y_train)

    rfc_train_preds = rfc_model.predict(X_train)
    rfc_test_preds = rfc_model.predict(X_test)

    train_acc_rfc = accuracy_score(y_train, rfc_train_preds)
    test_acc_rfc = accuracy_score(y_test, rfc_test_preds)
    
    train_f1_rfc = f1_score(y_train, rfc_train_preds)
    test_f1_rfc = f1_score(y_test, rfc_test_preds)

    train_conf_matrix_rfc = confusion_matrix(y_train, rfc_train_preds)
    train_true_pos = train_conf_matrix_rfc[0][0]
    train_false_pos = train_conf_matrix_rfc[0][1]
    train_false_neg = train_conf_matrix_rfc[1][0]
    train_true_neg = train_conf_matrix_rfc[1][1]
    
    test_conf_matrix_rfc = confusion_matrix(y_test, rfc_test_preds)
    test_true_pos = test_conf_matrix_rfc[0][0]
    test_false_pos = test_conf_matrix_rfc[0][1]
    test_false_neg = test_conf_matrix_rfc[1][0]
    test_true_neg = test_conf_matrix_rfc[1][1]   
    
    train_accuracy_list.append(train_acc_rfc)
    test_accuracy_list.append(test_acc_rfc)
    
    train_f1_list.append(train_f1_rfc)
    test_f1_list.append(test_f1_rfc)
    
    train_true_pos_list.append(train_true_pos)
    train_false_pos_list.append(train_false_pos)    
    train_false_neg_list.append(train_false_neg)
    train_true_neg_list.append(train_true_neg)
    
    test_true_pos_list.append(test_true_pos)
    test_false_pos_list.append(test_false_pos)    
    test_false_neg_list.append(test_false_neg)
    test_true_neg_list.append(test_true_neg)
    
rfc_random_state_df['train_accuracy'] = train_accuracy_list
rfc_random_state_df['test_accuracy'] = test_accuracy_list
rfc_random_state_df['train_f1_score'] = train_f1_list
rfc_random_state_df['test_f1_score'] = test_f1_list
rfc_random_state_df['train_true_pos'] = train_true_pos_list
rfc_random_state_df['train_false_pos'] = train_false_pos_list
rfc_random_state_df['train_false_neg'] = train_false_neg_list
rfc_random_state_df['train_true_neg'] = train_true_neg_list
rfc_random_state_df['test_true_pos'] = test_true_pos_list
rfc_random_state_df['test_false_pos'] = test_false_pos_list
rfc_random_state_df['test_false_neg'] = test_false_neg_list
rfc_random_state_df['test_true_neg'] = test_true_neg_list

In [39]:
rfc_random_state_df

,run_id,random_state,train_accuracy,test_accuracy,train_f1_score,test_f1_score,train_true_pos,train_false_pos,train_false_neg,train_true_neg,test_true_pos,test_false_pos,test_false_neg,test_true_neg
0,0,0,0.993057,0.988392,0.993009,0.988256,22974,0,319,22655,9735,0,226,9509
1,1,1,0.994722,0.989117,0.994694,0.989000,22831,0,241,22590,9876,2,213,9665
2,2,2,0.993061,0.987291,0.993012,0.987131,22913,0,318,22595,9793,3,246,9550
3,3,3,0.994394,0.990305,0.994363,0.990211,23013,0,258,22755,9695,1,187,9509
4,4,4,0.992553,0.987895,0.992497,0.987748,22961,0,342,22619,9747,1,235,9513
5,5,5,0.993212,0.988505,0.993166,0.988374,22835,0,310,22525,9872,2,225,9649
6,6,6,0.992906,0.988624,0.992856,0.988494,22908,0,325,22583,9800,1,222,9579
7,7,7,0.993080,0.987595,0.993032,0.987440,22834,0,316,22518,9874,1,244,9631
8,8,8,0.993139,0.989133,0.993091,0.989014,22955,0,315,22640,9753,1,211,9543
9,9,9,0.992761,0.988414,0.992708,0.988279,22999,0,333,22666,9709,1,224,9486


### XGBoost Classifier

In [40]:
xgb_random_state_df = pd.DataFrame(
    {'run_id': list(range(0, 10)),
     'random_state': list(range(0, 10))
    })

In [41]:
train_accuracy_list = []
test_accuracy_list = []
train_f1_list = []
test_f1_list = []
train_true_pos_list = []
train_false_pos_list = []
train_false_neg_list = []
train_true_neg_list = []
test_true_pos_list = []
test_false_pos_list = []
test_false_neg_list = []
test_true_neg_list = []


for j in range(10):
    
    shufled_agersens_df = full_upd_agersens_df.sample(frac=1, random_state=j).reset_index(drop=True)
    
    X = shufled_agersens_df.drop(columns=['target'])
    y = shufled_agersens_df['target']

    #split data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=j)

    #training part
    #combine them back for resampling
    train_data = pd.concat([X_train, y_train], axis=1)
    
    # separate minority and majority classes
    negative = train_data[train_data['target']==1]
    positive = train_data[train_data['target']==0]

    # downsample majority
    neg_downsampled = resample(negative,
     replace=True, # sample with replacement
     n_samples=len(positive), # match number in minority class
     random_state=j) # reproducible results
    # combine minority and downsampled majority
    train_downsampled = pd.concat([positive, neg_downsampled])

    #testing part
    #combine them back for resampling
    test_data = pd.concat([X_test, y_test], axis=1)
    
    # separate minority and majority classes
    negative = test_data[test_data['target']==1]
    positive = test_data[test_data['target']==0]

    # downsample majority
    neg_downsampled = resample(negative,
     replace=True, # sample with replacement
     n_samples=len(positive), # match number in minority class
     random_state=j) # reproducible results
    # combine minority and downsampled majority
    test_downsampled = pd.concat([positive, neg_downsampled])


    X_train = train_downsampled.drop(['target'],axis=1)

    X_test =  test_downsampled.drop(['target'],axis=1)

    y_train = train_downsampled['target']

    y_test = test_downsampled['target']

    xgb_model = xg.XGBClassifier()

    xgb_model.fit(X_train, y_train)

    xgb_train_preds = xgb_model.predict(X_train)
    xgb_test_preds = xgb_model.predict(X_test)

    train_acc_xgb = accuracy_score(y_train, xgb_train_preds)
    test_acc_xgb = accuracy_score(y_test, xgb_test_preds)
    
    train_f1_xgb = f1_score(y_train, xgb_train_preds)
    test_f1_xgb = f1_score(y_test, xgb_test_preds)

    train_conf_matrix_xgb = confusion_matrix(y_train, xgb_train_preds)
    train_true_pos = train_conf_matrix_xgb[0][0]
    train_false_pos = train_conf_matrix_xgb[0][1]
    train_false_neg = train_conf_matrix_xgb[1][0]
    train_true_neg = train_conf_matrix_xgb[1][1]
    
    test_conf_matrix_xgb = confusion_matrix(y_test, xgb_test_preds)
    test_true_pos = test_conf_matrix_xgb[0][0]
    test_false_pos = test_conf_matrix_xgb[0][1]
    test_false_neg = test_conf_matrix_xgb[1][0]
    test_true_neg = test_conf_matrix_xgb[1][1]   
    
    train_accuracy_list.append(train_acc_xgb)
    test_accuracy_list.append(test_acc_xgb)
    
    train_f1_list.append(train_f1_xgb)
    test_f1_list.append(test_f1_xgb)
    
    train_true_pos_list.append(train_true_pos)
    train_false_pos_list.append(train_false_pos)    
    train_false_neg_list.append(train_false_neg)
    train_true_neg_list.append(train_true_neg)
    
    test_true_pos_list.append(test_true_pos)
    test_false_pos_list.append(test_false_pos)    
    test_false_neg_list.append(test_false_neg)
    test_true_neg_list.append(test_true_neg)
    
xgb_random_state_df['train_accuracy'] = train_accuracy_list
xgb_random_state_df['test_accuracy'] = test_accuracy_list
xgb_random_state_df['train_f1_score'] = train_f1_list
xgb_random_state_df['test_f1_score'] = test_f1_list
xgb_random_state_df['train_true_pos'] = train_true_pos_list
xgb_random_state_df['train_false_pos'] = train_false_pos_list
xgb_random_state_df['train_false_neg'] = train_false_neg_list
xgb_random_state_df['train_true_neg'] = train_true_neg_list
xgb_random_state_df['test_true_pos'] = test_true_pos_list
xgb_random_state_df['test_false_pos'] = test_false_pos_list
xgb_random_state_df['test_false_neg'] = test_false_neg_list
xgb_random_state_df['test_true_neg'] = test_true_neg_list

/home/ml/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



[05:16:20] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ml/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/home/ml/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



[05:16:22] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ml/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/home/ml/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



...

/home/ml/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/home/ml/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



[05:16:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ml/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/home/ml/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



[05:16:40] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ml/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption



In [42]:
xgb_random_state_df

,run_id,random_state,train_accuracy,test_accuracy,train_f1_score,test_f1_score,train_true_pos,train_false_pos,train_false_neg,train_true_neg,test_true_pos,test_false_pos,test_false_neg,test_true_neg
0,0,0,1.0,0.997637,1.0,0.997633,22974,0,0,22974,9729,6,40,9695
1,1,1,1.0,0.998330,1.0,0.998328,22831,0,0,22831,9872,6,27,9851
2,2,2,1.0,0.998111,1.0,0.998109,22913,0,0,22913,9790,6,31,9765
3,3,3,1.0,0.997989,1.0,0.997986,23013,0,0,23013,9692,4,35,9661
4,4,4,1.0,0.998410,1.0,0.998408,22961,0,0,22961,9745,3,28,9720
5,5,5,1.0,0.998430,1.0,0.998428,22835,0,0,22835,9873,1,30,9844
6,6,6,1.0,0.998776,1.0,0.998775,22908,0,0,22908,9798,3,21,9780
7,7,7,1.0,0.998329,1.0,0.998327,22834,0,0,22834,9869,6,27,9848
8,8,8,1.0,0.998565,1.0,0.998563,22955,0,0,22955,9751,3,25,9729
9,9,9,1.0,0.998352,1.0,0.998350,22999,0,0,22999,9707,3,29,9681


### Neural Networks

In [43]:
nn_random_state_df = pd.DataFrame(
    {'run_id': list(range(0, 10)),
     'random_state': list(range(0, 10))
    })

In [44]:
train_accuracy_list = []
test_accuracy_list = []
train_f1_list = []
test_f1_list = []
train_true_pos_list = []
train_false_pos_list = []
train_false_neg_list = []
train_true_neg_list = []
test_true_pos_list = []
test_false_pos_list = []
test_false_neg_list = []
test_true_neg_list = []


for j in range(10):
    
    shufled_agersens_df = full_upd_agersens_df.sample(frac=1, random_state=j).reset_index(drop=True)
    
    X = shufled_agersens_df.drop(columns=['target'])
    y = shufled_agersens_df['target']

    #split data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=j)

    #training part
    #combine them back for resampling
    train_data = pd.concat([X_train, y_train], axis=1)
    
    # separate minority and majority classes
    negative = train_data[train_data['target']==1]
    positive = train_data[train_data['target']==0]

    # downsample majority
    neg_downsampled = resample(negative,
     replace=True, # sample with replacement
     n_samples=len(positive), # match number in minority class
     random_state=j) # reproducible results
    # combine minority and downsampled majority
    train_downsampled = pd.concat([positive, neg_downsampled])

    #testing part
    #combine them back for resampling
    test_data = pd.concat([X_test, y_test], axis=1)
    
    # separate minority and majority classes
    negative = test_data[test_data['target']==1]
    positive = test_data[test_data['target']==0]

    # downsample majority
    neg_downsampled = resample(negative,
     replace=True, # sample with replacement
     n_samples=len(positive), # match number in minority class
     random_state=j) # reproducible results
    # combine minority and downsampled majority
    test_downsampled = pd.concat([positive, neg_downsampled])


    X_train = train_downsampled.drop(['target'],axis=1)

    X_test =  test_downsampled.drop(['target'],axis=1)

    y_train = train_downsampled['target']

    y_test = test_downsampled['target']
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

    nn_model = Sequential()
    nn_model.add(Dense(128, activation='relu', input_shape=(len(X_train.columns),)))
    nn_model.add(Dense(64, activation='relu'))
    nn_model.add(Dense(32, activation='relu'))
    nn_model.add(Dense(1, activation='sigmoid'))
    
    nn_model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

    nn_model.fit(X_train, y_train, epochs=100, batch_size=10000, callbacks=[callback], verbose=1)
    
    # Predict the model
    nn_train_preds = nn_model.predict_classes(X_train)
    nn_test_preds = nn_model.predict_classes(X_test)

    train_acc_nn = accuracy_score(y_train, nn_train_preds)
    test_acc_nn = accuracy_score(y_test, nn_test_preds)
    
    train_f1_nn = f1_score(y_train, nn_train_preds)
    test_f1_nn = f1_score(y_test, nn_test_preds)

    train_conf_matrix_nn = confusion_matrix(y_train, nn_train_preds)
    train_true_pos = train_conf_matrix_nn[0][0]
    train_false_pos = train_conf_matrix_nn[0][1]
    train_false_neg = train_conf_matrix_nn[1][0]
    train_true_neg = train_conf_matrix_nn[1][1]
    
    test_conf_matrix_nn = confusion_matrix(y_test, nn_test_preds)
    test_true_pos = test_conf_matrix_nn[0][0]
    test_false_pos = test_conf_matrix_nn[0][1]
    test_false_neg = test_conf_matrix_nn[1][0]
    test_true_neg = test_conf_matrix_nn[1][1]   
    
    train_accuracy_list.append(train_acc_nn)
    test_accuracy_list.append(test_acc_nn)
    
    train_f1_list.append(train_f1_nn)
    test_f1_list.append(test_f1_nn)
    
    train_true_pos_list.append(train_true_pos)
    train_false_pos_list.append(train_false_pos)    
    train_false_neg_list.append(train_false_neg)
    train_true_neg_list.append(train_true_neg)
    
    test_true_pos_list.append(test_true_pos)
    test_false_pos_list.append(test_false_pos)    
    test_false_neg_list.append(test_false_neg)
    test_true_neg_list.append(test_true_neg)
    
nn_random_state_df['train_accuracy'] = train_accuracy_list
nn_random_state_df['test_accuracy'] = test_accuracy_list
nn_random_state_df['train_f1_score'] = train_f1_list
nn_random_state_df['test_f1_score'] = test_f1_list
nn_random_state_df['train_true_pos'] = train_true_pos_list
nn_random_state_df['train_false_pos'] = train_false_pos_list
nn_random_state_df['train_false_neg'] = train_false_neg_list
nn_random_state_df['train_true_neg'] = train_true_neg_list
nn_random_state_df['test_true_pos'] = test_true_pos_list
nn_random_state_df['test_false_pos'] = test_false_pos_list
nn_random_state_df['test_false_neg'] = test_false_neg_list
nn_random_state_df['test_true_neg'] = test_true_neg_list

Epoch 1/100
5/5 [==============================] - 0s 2ms/step - loss: 4.4278 - accuracy: 0.4243
Epoch 2/100
5/5 [==============================] - 0s 2ms/step - loss: 2.4928 - accuracy: 0.5388
Epoch 3/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4684 - accuracy: 0.8401
Epoch 4/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5981 - accuracy: 0.6189
Epoch 5/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4520 - accuracy: 0.8162
Epoch 6/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4355 - accuracy: 0.7818
Epoch 7/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4731 - accuracy: 0.6840
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4340 - accuracy: 0.8031
Epoch 9/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4023 - accuracy: 0.8573
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4523 - accuracy: 0.7921
Epoch 11/100
5/5 [===========

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).



Epoch 1/100
5/5 [==============================] - 0s 2ms/step - loss: 8.0689 - accuracy: 0.5035
Epoch 2/100
5/5 [==============================] - 0s 2ms/step - loss: 0.6475 - accuracy: 0.5594
Epoch 3/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5249 - accuracy: 0.7555
Epoch 4/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5614 - accuracy: 0.5585
Epoch 5/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5396 - accuracy: 0.6613
Epoch 6/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4732 - accuracy: 0.8182
Epoch 7/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5758 - accuracy: 0.6205
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4199 - accuracy: 0.8806
Epoch 9/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4664 - accuracy: 0.7484
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 0.3761 - accuracy: 0.8968
Epoch 11/100
5/5 [===========

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).



Epoch 1/100
5/5 [==============================] - 0s 2ms/step - loss: 7.2354 - accuracy: 0.5145
Epoch 2/100
5/5 [==============================] - 0s 1ms/step - loss: 1.7607 - accuracy: 0.5403
Epoch 3/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5180 - accuracy: 0.7809
Epoch 4/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4668 - accuracy: 0.8275
Epoch 5/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4281 - accuracy: 0.8347
Epoch 6/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4244 - accuracy: 0.8420
Epoch 7/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5633 - accuracy: 0.5864
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4781 - accuracy: 0.7922
Epoch 9/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4198 - accuracy: 0.8717
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4846 - accuracy: 0.7716
Epoch 11/100
5/5 [===========

...

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).



Epoch 1/100
5/5 [==============================] - 0s 2ms/step - loss: 6.2079 - accuracy: 0.5043
Epoch 2/100
5/5 [==============================] - 0s 1ms/step - loss: 0.6007 - accuracy: 0.5913
Epoch 3/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4244 - accuracy: 0.8287
Epoch 4/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4527 - accuracy: 0.7873
Epoch 5/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4373 - accuracy: 0.8130
Epoch 6/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4552 - accuracy: 0.8104
Epoch 7/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4345 - accuracy: 0.8140
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 0.3940 - accuracy: 0.8764
Epoch 9/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4059 - accuracy: 0.8336
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 0.3869 - accuracy: 0.8773
Epoch 11/100
5/5 [===========

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).



Epoch 1/100
5/5 [==============================] - 0s 2ms/step - loss: 9.4062 - accuracy: 0.5023
Epoch 2/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5868 - accuracy: 0.6457
Epoch 3/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4776 - accuracy: 0.8204
Epoch 4/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4507 - accuracy: 0.8479
Epoch 5/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5479 - accuracy: 0.6596
Epoch 6/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4054 - accuracy: 0.8840
Epoch 7/100
5/5 [==============================] - 0s 1ms/step - loss: 0.3664 - accuracy: 0.8689
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5180 - accuracy: 0.6945
Epoch 9/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5281 - accuracy: 0.7769
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 0.3395 - accuracy: 0.8830
Epoch 11/100
5/5 [===========

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).

/home/ml/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning:

`model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).



In [45]:
nn_random_state_df

,run_id,random_state,train_accuracy,test_accuracy,train_f1_score,test_f1_score,train_true_pos,train_false_pos,train_false_neg,train_true_neg,test_true_pos,test_false_pos,test_false_neg,test_true_neg
0,0,0,0.882715,0.880996,0.892988,0.891815,18074,4900,489,22485,7603,2132,185,9550
1,1,1,0.937191,0.937740,0.936071,0.936663,21797,1034,1834,20997,9431,447,783,9095
2,2,2,0.862305,0.864996,0.877127,0.879416,16994,5919,391,22522,7302,2494,151,9645
3,3,3,0.935363,0.937345,0.933837,0.935989,22056,957,2018,20995,9294,402,813,8883
4,4,4,0.919102,0.919009,0.922847,0.922874,19989,2972,743,22218,8470,1278,301,9447
5,5,5,0.888877,0.890267,0.898131,0.899308,18223,4612,463,22372,7904,1970,197,9677
6,6,6,0.879278,0.879604,0.890304,0.890639,17840,5068,463,22445,7632,2169,191,9610
7,7,7,0.931440,0.933114,0.928917,0.930899,22079,755,2376,20458,9531,344,977,8898
8,8,8,0.872816,0.872668,0.885395,0.885340,17516,5439,400,22555,7434,2320,164,9590
9,9,9,0.885778,0.889238,0.895700,0.898475,18184,4815,439,22560,7751,1959,192,9518


From the above experiments, I can infer that `XGBoost Classifier` performs the best out of all the models with `Random Forest Classifier` coming in a close second. I think `SVM` and `Neural Networks` does a decent enough job in classifying the data. 

Downsampling the data really did help in the prediction of `Non-Spurious` data. The `Neural Networks` performed better, I was able to run `SVM` with less data. Both `XGBoost Classifier`and `Random Forest Classifier` does a much better job and there is also a huge reduction in the values for False Positives and False Negatives.